In [ ]:
import pandas as pd
import numpy as np
import torch
from PIL import Image
from tqdm.auto import tqdm

# Настройки
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 32

# Модели (можно легко заменить на другие с HuggingFace)
TEXT_MODEL_NAME = 'all-MiniLM-L6-v2'  # Легкая и быстрая модель для текстов
IMAGE_MODEL_NAME = 'openai/clip-vit-base-patch32' # CLIP от OpenAI

print(f"Using device: {DEVICE}")

Обработка Текста (Text Embeddings)
Используем библиотеку sentence-transformers. Она работает проще и часто лучше, чем "голый" BERT, так как выдает уже нормализованные векторы предложений.

In [ ]:
from sentence_transformers import SentenceTransformer

def get_text_embeddings(texts, model_name, device):
    print("Loading Text Model...")
    model = SentenceTransformer(model_name, device=device)
    
    print("Encoding texts...")
    # show_progress_bar=True покажет полоску прогресса
    embeddings = model.encode(texts, batch_size=BATCH_SIZE, show_progress_bar=True)
    
    return embeddings

# Вызов
text_features = get_text_embeddings(
    texts=df['text_col'].tolist(), 
    model_name=TEXT_MODEL_NAME, 
    device=DEVICE
)

print(f"Text embeddings shape: {text_features.shape}")
# Обычно это (N_samples, 384) для MiniLM

Обработка Изображений (CLIP / Vision Transformer)
Здесь мы используем transformers от HuggingFace. Мы берем только визуальную часть CLIP (CLIPVisionModel), чтобы превратить картинку в вектор.

In [2]:
from transformers import CLIPProcessor, CLIPVisionModel
from torch.utils.data import DataLoader, Dataset

class ImageDataset(Dataset):
    def __init__(self, image_paths, processor):
        self.image_paths = image_paths
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Открываем картинку
        path = self.image_paths[idx]
        image = Image.open(path).convert("RGB")
        # Процессор сам делает ресайз и нормализацию
        return self.processor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)

def get_image_embeddings(image_paths, model_name, device):
    print("Loading Image Model...")
    # Загружаем только Vision часть CLIP
    model = CLIPVisionModel.from_pretrained(model_name).to(device)
    processor = CLIPProcessor.from_pretrained(model_name)
    
    dataset = ImageDataset(image_paths, processor)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    all_embeddings = []
    model.eval()
    
    print("Encoding images...")
    with torch.no_grad():
        for batch in tqdm(loader):
            batch = batch.to(device)
            # Прогоняем через модель
            outputs = model(batch)
            # pooler_output - это вектор, представляющий всю картинку
            # Для CLIP vit-base это вектор размером 768
            embeds = outputs.pooler_output.cpu().numpy()
            all_embeddings.append(embeds)
            
    return np.vstack(all_embeddings)

# Вызов
image_features = get_image_embeddings(
    image_paths=df['image_path'].tolist(), 
    model_name=IMAGE_MODEL_NAME, 
    device=DEVICE
)

print(f"Image embeddings shape: {image_features.shape}")
# Обычно это (N_samples, 768)

ModuleNotFoundError: No module named 'transformers'

Обработка Табличных данных (Tabular Preprocessing)
Классическая обработка: числа шкалируем, категории кодируем.

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Определяем колонки
numeric_features = ['price']
categorical_features = ['category']

# Создаем простой препроцессор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(sparse_output=False), categorical_features)
    ])

# Обучаем и трансформируем
tabular_features = preprocessor.fit_transform(df)

print(f"Tabular features shape: {tabular_features.shape}")

NameError: name 'df' is not defined

Слияние (Fusion)
Самый важный этап. Мы просто "склеиваем" (concatenation) все векторы в одну длинную строку для каждого примера.
code
Python


In [ ]:
# Объединяем всё в одну матрицу: [Текст | Картинки | Таблица]
X_final = np.hstack([
    text_features,    # например, 384 колонки
    image_features,   # например, 768 колонок
    tabular_features  # например, 4 колонки
])

y_final = df['target'].values

print(f"FINAL Matrix shape: {X_final.shape}")
# Результат: (100, 384 + 768 + 4) = (100, 1156)